In [ ]:
!pip install psycopg2-binary
!pip install ipython-sql
!pip install pandas

In [16]:
import configparser

In [26]:
config = configparser.ConfigParser()
config.read('config/db.cfg')
pg_info = config['POSTGRES']

dbname = pg_info['db']
host = pg_info['host']
user = pg_info['user']
password = pg_info['password']
port = ''
if 'port' in pg_info:
    port = ':' + pg_info['port']

conn_string = 'postgresql://{}:{}@{}{}/{}'.format(user, password, host, port, dbname)

In [27]:
%reload_ext sql

In [28]:
%sql $conn_string

'Connected: postgres@mtatransit'

In [ ]:
%%sql
SELECT station, recorded_date, sum(entry_count) as daily_entry
FROM turnstile_transformed
group by station, recorded_date
order by recorded_date desc
LIMIT 50

In [ ]:
%%sql
SELECT recorded_date, sum(entry_count) as daily_entry
FROM turnstile_transformed
where entry_count < 2048
group by recorded_date
order by recorded_date desc
LIMIT 365

In [90]:
%%sql
SELECT 
    extract(year from recorded_date) as year,
    avg(daily_entry) as daily_entry_avg_weekdays
FROM 
(
    SELECT 
        recorded_date, 
        sum(entry_count) as daily_entry
    FROM 
        turnstile_transformed
    where 
        entry_count < 2048
    group by 
        recorded_date
    having 
        extract(isodow from recorded_date)  <> 6 and extract(isodow from recorded_date) <> 7 and extract(year from recorded_date) >= 2017
) as f
GROUP BY 
    year
order by 
    year desc

 * postgresql://postgres:***@db/mtatransit
4 rows affected.


year,avg
2020.0,5147265.375000000000
2019.0,5366963.624521072797
2018.0,5310282.022988505747
2017.0,5444626.134615384615


In [91]:
%%sql
SELECT 
    extract(year from recorded_date) as year,
    avg(daily_entry) as daily_entry_avg_saturdays
FROM 
(
    SELECT 
        recorded_date, 
        sum(entry_count) as daily_entry
    FROM 
        turnstile_transformed
    where 
        entry_count < 2048
    group by 
        recorded_date
    having 
        extract(isodow from recorded_date) = 6 and extract(year from recorded_date) >= 2017
) as f
GROUP BY 
    year
order by 
    year desc

 * postgresql://postgres:***@db/mtatransit
4 rows affected.


year,daily_entry_avg_saturdays
2020.0,2861537.111111111111
2019.0,3210772.980769230769
2018.0,3177205.288461538462
2017.0,3286764.442307692308


In [92]:
%%sql
SELECT 
    extract(year from recorded_date) as year,
    avg(daily_entry) as daily_entry_avg_sundays
FROM 
(
    SELECT 
        recorded_date, 
        sum(entry_count) as daily_entry
    FROM 
        turnstile_transformed
    where 
        entry_count < 2048
    group by 
        recorded_date
    having 
        extract(isodow from recorded_date) = 7 and extract(year from recorded_date) >= 2017
) as f
GROUP BY 
    year
order by 
    year desc

 * postgresql://postgres:***@db/mtatransit
4 rows affected.


year,daily_entry_avg_sundays
2020.0,2327390.111111111111
2019.0,2549091.000000000000
2018.0,2525166.788461538462
2017.0,2653569.792452830189
